In [1]:
from transformers import pipeline
embedder = pipeline("feature-extraction", model="roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
x=embedder('Its good but the adds to get more blocks wont')
len(x[0][0])

768

In [3]:
import pandas as pd
import numpy as np
import time
import os
import re
from tqdm.notebook import tqdm
from tqdm import tnrange
from torch.utils.data import Dataset,random_split
import csv
import tqdm
import emoji
import math

In [ ]:
# 載入資料集 ALL_ORIGIN_DATA_review.csv 僅:index/評論/appID/userID
ALL_ORIGIN_DATA_review = pd.read_csv('./Data/ALL_ORIGIN_DATA_review.csv')

In [5]:
### 先做分詞，以便計算每則評論長度。分詞完存在 reviews_lst(list)
reviews_lst=[]
for count,review in enumerate(ALL_ORIGIN_DATA_review['review']):
     
    # 刪掉emoji
    sent = emoji.demojize(review)  
    sent = re.sub(':\S+?:', ' ', sent)
    sent = re.sub(r'[^a-zA-Z ]', '', sent)
    # 轉小寫
    sent = sent.lower()     
    reviews_lst.append(sent)

In [6]:
len(reviews_lst)

1116196

In [7]:
### 切出每個review的前300個字 (因模型限制token後總長度數需<=512)
split_300_reviews_lst = []
for index,review in enumerate(reviews_lst):
    if len(review)>300:
        review = review.split()[:300]
        review = ''.join( e+" " for e in review )
    split_300_reviews_lst.append(review)
print(len(split_300_reviews_lst))

1116196


In [8]:
from pandas.core.frame import DataFrame
ALL_ORIGIN_DATA_review = {"review_index":list(ALL_ORIGIN_DATA_review['review_index']), "userId":list(ALL_ORIGIN_DATA_review['userId']),"appId":list(ALL_ORIGIN_DATA_review['appId']),"roberta": split_300_reviews_lst}
ALL_ORIGIN_DATA_review = DataFrame(ALL_ORIGIN_DATA_review)
ALL_ORIGIN_DATA_review.head(5)

,review_index,userId,appId,roberta
0,0,0,5,now i understand why so many low marks two obj...
1,1,1,5,its good but the adds to get more blocks wont ...
2,2,2,5,i love the game my lil sis showed me it and iv...
3,3,3,5,good game it deserves stars because it isnt a...
4,4,4,5,dear people i lovvvvvvvve it but why do u have...


In [9]:
ALL_ORIGIN_DATA_review.shape

(1116196, 4)

### 接下來把所有轉RoBERTa的任務分成三段跑，否則會有memory不足
#### 若因設備限制能無法在此檔案中接續run
#### 可以改在多個檔案分開跑，再合併所有輸出即可

In [11]:
# 1) index=0-39w9999
from tqdm import tnrange
index=0
gap_num = 100
lst = []

for i in tnrange(0,400000,gap_num):
  
    if i + gap_num < ALL_ORIGIN_DATA_review.shape[0]:
        
        end = i + gap_num
   
        lst.append(ALL_ORIGIN_DATA_review[i:end]['roberta'].apply(lambda x:embedder(x)[0][0])) 
        
    else:
        lst.append(ALL_ORIGIN_DATA_review[i:]['roberta'].apply(lambda x:embedder(x)[0][0]))
        
    index += 1

C:\Users\user\anaconda3\envs\pytorch1\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/4000 [00:00<?, ?it/s]

In [12]:
embedding = np.array(lst).reshape(-1)
dfz = pd.DataFrame(embedding,columns=['roberta_embedding'])

In [13]:
# 2) index=40w-799999　
index=0
gap_num = 100
lst_2 = []
# for i in tnrange(0,ALL_ORIGIN_DATA_review.shape[0],gap_num):
for i in tnrange(400000,800000,gap_num):
  
    if i + gap_num < ALL_ORIGIN_DATA_review.shape[0]:
        
        end = i + gap_num
   
        lst_2.append(ALL_ORIGIN_DATA_review[i:end]['roberta'].apply(lambda x:embedder(x)[0][0]))  
        
    else:
        lst_2.append(ALL_ORIGIN_DATA_review[i:]['roberta'].apply(lambda x:embedder(x)[0][0]))
        
    index += 1

C:\Users\user\anaconda3\envs\pytorch1\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  import sys


  0%|          | 0/4000 [00:00<?, ?it/s]

In [14]:
embedding_2 = np.array(lst_2).reshape(-1)
dfz_2 = pd.DataFrame(embedding_2,columns=['roberta_embedding'])

In [15]:
# 把前40萬和中間40萬資料合併
# ignore_index = True 可以忽略合併時舊的 index 欄位，改採用自動產生的 index

df_12 = pd.concat([dfz,dfz_2],axis=0, ignore_index=True)
df_12

,roberta_embedding
0,"[-0.05433040112257004, 0.04897385835647583, -0..."
1,"[-0.030177436769008636, 0.04714367538690567, -..."
2,"[-0.059469737112522125, 0.0544145405292511, -0..."
3,"[-0.048043347895145416, 0.009005382657051086, ..."
4,"[-0.027240321040153503, 0.051726289093494415, ..."
...,...
799995,"[-0.023773647844791412, 0.03545856475830078, -..."
799996,"[-0.03653782606124878, 0.05648522451519966, -0..."
799997,"[-0.03418271988630295, 0.07178327441215515, -0..."
799998,"[-0.0356803834438324, 0.05631208047270775, -0...."


In [16]:
(df_12['roberta_embedding'][0])

[-0.05433040112257004,
 0.04897385835647583,
 -0.020001187920570374,
 -0.08130734413862228,
 0.03175841271877289,
 -0.09207761287689209,
 -0.008430570363998413,
 0.04394415393471718,
 0.03771510720252991,
 -0.02710893750190735,
 -0.008781764656305313,
 0.058293960988521576,
 0.012238509953022003,
 0.03327557072043419,
 0.08090201765298843,
 0.01141715794801712,
 -0.019149329513311386,
 0.011759355664253235,
 0.005239762365818024,
 -0.011730071157217026,
 -0.1304076761007309,
 0.02531876042485237,
 -0.05226752907037735,
 0.10842496156692505,
 0.01398574560880661,
 0.02978977933526039,
 0.05870471149682999,
 0.08086755871772766,
 -0.07186675071716309,
 -0.02047938108444214,
 -0.04982173442840576,
 -0.0347360298037529,
 0.005555186420679092,
 -0.025034628808498383,
 0.05278247222304344,
 0.046441107988357544,
 0.07334405183792114,
 -0.018275074660778046,
 -0.12362617254257202,
 0.04797561094164848,
 -0.014612771570682526,
 -0.042198795825242996,
 0.026814885437488556,
 -0.0079547874629497

In [ ]:
# 3) index=80w-end　
index=0
gap_num = 100
lst_3 = []

for i in tnrange(800000,1116196,gap_num):
  
    if i + gap_num < ALL_ORIGIN_DATA_review.shape[0]:
        
        end = i + gap_num
   
        lst_3.append(ALL_ORIGIN_DATA_review[i:end]['roberta'].apply(lambda x:embedder(x)[0][0]))  
        
    else:
        lst_3.append(ALL_ORIGIN_DATA_review[i:]['roberta'].apply(lambda x:embedder(x)[0][0]))
        
    index += 1

In [ ]:
lst_3_new = []
for i in lst_3:
    testttttttt = i.values.reshape(-1)
    for j in testttttttt:
        j = list(j)
        j_list = []
        j_list.append(j)
        lst_3_new.append(j_list)
print(len(lst_3_new))

In [74]:
dfz_3 = pd.DataFrame(lst_3_new,columns=['roberta_embedding'])

In [61]:
type(df_12['roberta_embedding'][0][0])

float

In [62]:
(dfz_3['roberta_embedding'][0][0])

'['

In [56]:
type(dfz_3['roberta_embedding'][0][0])

float

In [63]:
for indx, i in enumerate(dfz_3['roberta_embedding']):
    dfz_3['roberta_embedding'][indx]=i.replace('[','').replace(']','').split(',')

    for indx_j,j in enumerate(dfz_3['roberta_embedding'][indx]):
        dfz_3['roberta_embedding'][indx][indx_j] = float(j)

C:\Users\user\anaconda3\envs\pytorch1\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
df_123 = pd.concat([df_12,dfz_3],axis=0, ignore_index=True)
df_123

,roberta_embedding,Unnamed: 0
0,"[-0.05433040112257004, 0.04897385835647583, -0...",NaN
1,"[-0.030177436769008636, 0.04714367538690567, -...",NaN
2,"[-0.059469737112522125, 0.0544145405292511, -0...",NaN
3,"[-0.048043347895145416, 0.009005382657051086, ...",NaN
4,"[-0.027240321040153503, 0.051726289093494415, ...",NaN
...,...,...
1116191,"[-0.03555864095687866, 0.06557829678058624, -0...",316191.0
1116192,"[-0.030482061207294464, 0.02613111212849617, -...",316192.0
1116193,"[-0.04347474128007889, 0.06888260692358017, -0...",316193.0
1116194,"[-0.010704051703214645, 0.03202276676893234, -...",316194.0


####  串起上面分三組得到的RoBERTa embedding後，
#### 存到 ALL_ORIGIN_DATA_review_roberta_0toend_embedding

In [65]:
import pickle
with open('./Data/ALL_ORIGIN_DATA_review_roberta_0toend_embedding', 'wb') as f:
    pickle.dump(df_123, f)